In [37]:
#All the import statements goes here
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib as plt
import os
import pickle
import time
from keras import backend as K

path1 = "J:/SF/Thermal Load Prediction/Repos folders/loadPrediction/"
path_list = ["J:/SF/Thermal Load Prediction/Datas/Open Power System Data/opsd-household_data-2017-09-01/",
             "/Volumes/ADITHYA/Adithya/ML materials/DATA SETS/Open Power System /opsd-household_data-2017-09-01/"]
path_sd = ["/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/"]
path_sfroot = ["/home/sfroot/loadPrediction/"]

if("adithya" in os.getlogin()):
    path = path_list[1]
elif("siddharth" in os.getlogin()):
    path = path_sd[0]
elif("sfroot" in os.getlogin()):    
    path = path_sfroot[0]
else:
    path=path1

In [3]:
# read1 contains all rows and columns from csv file
read1 = pd.read_csv(path+"household_data_1min_singleindex.csv")

# read will have columns with load data
read = read1[read1.columns[2:11]]
x = np.array(read.values)

# t is to find out which minute in the data has minimum occurances of NaN
t = np.zeros(shape=(60))

for i in range(60):
    j=i;
    c=0
    while(j <= x.shape[0]-1):
        c1=0
        for k in x[j]:
            if(np.isnan(k)):
                c1+=1
        if(c1!=0):
            c+=1
        j+=60
    t[i]=c
    
# amin(t) gives minimum count and argmin(t)+1 gives minute value
print(np.amin(t))
print(np.argmin(t)+1)

FileNotFoundError: File b'/home/sfroot/loadPrediction/household_data_1min_singleindex.csv' does not exist

In [3]:
# using 1hr data
hr_data = pd.read_csv(path+"household_data_60min_singleindex.csv")
# removing the following columns
del hr_data["interpolated_values"]
del hr_data["DE_KN_residential4_grid_export"]
del hr_data["DE_KN_residential4_pv"]
del hr_data["cet_cest_timestamp"]

# yr_data will have the data from 2016-01-01 to 2017-01-01
yr_data = pd.DataFrame(hr_data.values[2137:10921])
yr_data.columns = hr_data.columns

In [4]:
yr_data.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2016-01-01T02:00:00Z,25.092,203.147,35.935,1146.79,798.665,29.125,16.217
1,2016-01-01T03:00:00Z,25.092,203.147,35.949,1147.79,799.32,29.125,16.217
2,2016-01-01T04:00:00Z,25.092,203.147,35.963,1148.54,799.855,29.125,16.217
3,2016-01-01T05:00:00Z,25.092,203.147,35.982,1149.23,800.335,29.172,16.217
4,2016-01-01T06:00:00Z,25.092,203.147,36.002,1150.04,800.995,29.172,16.217


In [5]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap
df = pd.DataFrame()
for i in range(len(yr_data.columns)):
    if(i != 0):
        df = yr_data.iloc[:,i]
        flag1 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                grad = (df.values[j] - df.values[prev])/(j-prev)
                for k in range(prev+1, j):
                    df.values[k] = df.values[k-1] + grad
                flag1 = False
        yr_data[yr_data.columns[i]] = df.values

In [6]:
yr_data.isnull().any()

utc_timestamp                         False
DE_KN_residential4_dishwasher         False
DE_KN_residential4_ev                 False
DE_KN_residential4_freezer            False
DE_KN_residential4_grid_import        False
DE_KN_residential4_heat_pump          False
DE_KN_residential4_refrigerator       False
DE_KN_residential4_washing_machine    False
dtype: bool

In [7]:
# exporting the new non null containing data to new csv
yr_data.to_csv("Year_Data_New.csv",sep=",",encoding="utf-8")

In [8]:
1.0*381/np.sum(hr_data.isnull().sum())

0.08397619572404673

In [9]:
hr_data.isnull().sum()

utc_timestamp                            0
DE_KN_residential4_dishwasher         1083
DE_KN_residential4_ev                  405
DE_KN_residential4_freezer             793
DE_KN_residential4_grid_import         343
DE_KN_residential4_heat_pump           298
DE_KN_residential4_refrigerator       1234
DE_KN_residential4_washing_machine     381
dtype: int64

In [10]:
hr_data.shape

(11867, 8)

In [11]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap on hr data
df = pd.DataFrame()
for i in range(len(hr_data.columns)):
    if(i != 0):
        df = hr_data.iloc[:,i]
        flag1 = False
        flag2 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                if(j==0):
                    flag2 = True
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                if(flag2):
                    flag2 = False
                    for k in range(prev+1,j):
                        df.values[k] = df.values[j]
                else:
                    grad = (df.values[j] - df.values[prev])/(j-prev)
                    for k in range(prev+1, j):
                        df.values[k] = df.values[k-1] + grad
                flag1 = False
        hr_data[hr_data.columns[i]] = df.values

In [12]:
hr_data.shape

(11867, 8)

In [13]:
hr_data.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2015-10-04T01:00:00Z,0.002,0.945,0.012,0.077,0.00,0.0,0.001
1,2015-10-04T02:00:00Z,0.002,0.945,0.012,0.077,0.02,0.0,0.001
2,2015-10-04T03:00:00Z,0.002,0.945,0.012,0.077,0.03,0.0,0.001
3,2015-10-04T04:00:00Z,0.002,0.945,0.012,0.077,0.04,0.0,0.001
4,2015-10-04T05:00:00Z,0.002,0.945,0.012,0.077,0.06,0.0,0.001


In [14]:
# counted null values after interpolation in entire dataset

hr_data = hr_data.iloc[:11012,:]

nan_cnt = []

nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_dishwasher'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_ev'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_freezer'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_grid_import'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_heat_pump'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_refrigerator'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_washing_machine'])))

print(nan_cnt)

[0, 0, 0, 0, 0, 0, 0]


In [15]:
#Another efficient way to check if there are any NULLS present in any of the columns.
hr_data.isnull().sum(axis=0)

utc_timestamp                         0
DE_KN_residential4_dishwasher         0
DE_KN_residential4_ev                 0
DE_KN_residential4_freezer            0
DE_KN_residential4_grid_import        0
DE_KN_residential4_heat_pump          0
DE_KN_residential4_refrigerator       0
DE_KN_residential4_washing_machine    0
dtype: int64

In [16]:
hr_data.to_csv("Hour_Data_New.csv",sep=",",encoding="utf-8")
#hr_data = pd.read_csv(path1+"Hour_Data_New.csv")
#del hr_data['Unnamed: 0']

In [52]:
'''
Function that takes in a dataframe and then returns the dictionary of accuracies.
'''

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def nn_model(localdf,localdict,times):
    for counter in range(times):
        for i in range(len(localdf.columns)):
            df = pd.DataFrame()
            if(i != 0):
                print(localdf.columns[i])
                df = localdf[localdf.columns[i]]
                ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
                ndf.columns = ('t-2', 't-1', 't')
                ndf = ndf.iloc[2:]

                x = ndf[ndf.columns[0:2]]
                y = ndf[ndf.columns[-1:]]

                X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
                model = Sequential()
                model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
                #Output Layer.
                model.add(Dense(1,kernel_initializer='normal'))
                #Compile model.
                model.compile(loss='mean_squared_error',optimizer='adam',)
                #Fit the model.
                model.fit(X_train,y_train,epochs=200,batch_size=200,verbose=1)
                #Final Evaluation of the model.
                scores = model.predict(X_test)
                #print (coeff_determination(y_test,scores))
            
                print (scores[:100],y_test[:100])
                print ("Baseline Error : %.5f" % (scores[1]))
                localdict.setdefault(counter,[]).append((scores[1]))
                model.summary()
    return localdict            

In [18]:
'''
Function that computes the actual load consumed, rather than the meter readings.
'''
def actual_load(df):
    data = []
    temp = np.zeros((1,7))
    tempdate = []
    
    print (df.shape)
    
    for i in range(df.shape[0]-1,1,-1):
        if((i%1000)==0):
            print (i)
        temp = df.values[i,1:] - df.values[i-1,1:]  
        tempdate.append(df.values[i,0])
        data.append(temp)
    
    tempdate = np.array(tempdate)
    return tempdate,data        

In [30]:
hr_data_copy = hr_data
tempdate,data = actual_load(hr_data_copy)

(11012, 8)
11000
10000
9000
8000
7000
6000
5000
4000
3000
2000
1000


In [31]:
#To chage the dimension in order to perform hstack
tempdate = tempdate.reshape((tempdate.shape[0],-1))
data = np.hstack((tempdate,data))
#To reverse the array
data = data[::-1]
np.shape(data)

(11010, 8)

In [32]:
new_df = pd.DataFrame(data,columns=hr_data.columns)
print (new_df.shape,hr_data.shape)

(11010, 8) (11012, 8)


In [33]:
hr_data.tail()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
11007,2017-01-04T16:00:00Z,117.198,1229.077,199.749,5177.600,3477.680,191.874,134.062
11008,2017-01-04T17:00:00Z,117.199,1229.077,199.757,5178.388,3478.287,191.886,134.062
11009,2017-01-04T18:00:00Z,117.199,1229.077,199.766,5179.642,3479.360,191.912,134.062
11010,2017-01-04T19:00:00Z,117.199,1229.077,199.781,5180.477,3479.945,191.912,134.062
11011,2017-01-04T20:00:00Z,117.199,1229.077,199.796,5181.974,3481.030,191.951,134.062


In [34]:
new_df.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2015-10-04T03:00:00Z,0,0,0,0,0.01,0,0
1,2015-10-04T04:00:00Z,0,0,0,0,0.01,0,0
2,2015-10-04T05:00:00Z,0,0,0,0,0.02,0,0
3,2015-10-04T06:00:00Z,0,0,0,0,0.01,0,0
4,2015-10-04T07:00:00Z,0,0,0,0,0.01,0,0


TypeError: '<=' not supported between instances of 'str' and 'float'

In [53]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on the year 2016-01-01 to 2017-01-01 data

#acc_dict1={}

acc={}
ti = time.time()
acc = nn_model(new_df,acc,1)
tf = time.time()
print ("Time Taken : ",tf-ti)
print (acc)

DE_KN_residential4_dishwasher
Epoch 1/200
8806/8806 [==============================] - 0s - loss: 0.0050     
Epoch 2/200
8806/8806 [==============================] - 0s - loss: 0.0050     
Epoch 3/200
8806/8806 [==============================] - 0s - loss: 0.0050     
Epoch 4/200
8806/8806 [==============================] - 0s - loss: 0.0050     
Epoch 5/200
8806/8806 [==============================] - 0s - loss: 0.0049     
Epoch 6/200
8806/8806 [==============================] - 0s - loss: 0.0049     
Epoch 7/200
8806/8806 [==============================] - 0s - loss: 0.0049     
Epoch 8/200
8806/8806 [==============================] - 0s - loss: 0.0049     
Epoch 9/200
8806/8806 [==============================] - 0s - loss: 0.0048     
Epoch 10/200
8806/8806 [==============================] - 0s - loss: 0.0048     
Epoch 11/200
8806/8806 [==============================] - 0s - loss: 0.0048     
Epoch 12/200
8806/8806 [==============================] - 0s - loss: 0.0048     
Epoch 1

8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 102/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 103/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 104/200
8806/8806 [==============================] - 0s - loss: 0.0044       
Epoch 105/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 106/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 107/200
8806/8806 [==============================] - 0s - loss: 0.0044       
Epoch 108/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 109/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 110/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 111/200
8806/8806 [==============================] - 0s - loss: 0.0044     
Epoch 112/200
8806/8806 [==============================] - 0s - loss: 0.0044       
Epoch 113/200
8806/8806 

8806/8806 [==============================] - 0s - loss: 0.0042       
[[ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00608182]
 [ 0.00998487]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00672102]
 [ 0.00733   ]
 [ 0.00558546]
 [ 0.01026817]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.01102572]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00998487]
 [ 0.00579853]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00733   ]
 [ 0.00917786]


8806/8806 [==============================] - 0s - loss: 0.0844     
Epoch 65/200
8806/8806 [==============================] - 0s - loss: 0.0844     
Epoch 66/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 67/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 68/200
8806/8806 [==============================] - 0s - loss: 0.0844     
Epoch 69/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 70/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 71/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 72/200
8806/8806 [==============================] - 0s - loss: 0.0844     
Epoch 73/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 74/200
8806/8806 [==============================] - 0s - loss: 0.0843     
Epoch 75/200
8806/8806 [==============================] - 0s - loss: 0.0842     
Epoch 76/200
8806/8806 [=================

8806/8806 [==============================] - 0s - loss: 0.0834     
Epoch 164/200
8806/8806 [==============================] - 0s - loss: 0.0833     
Epoch 165/200
8806/8806 [==============================] - 0s - loss: 0.0833     
Epoch 166/200
8806/8806 [==============================] - 0s - loss: 0.0833     
Epoch 167/200
8806/8806 [==============================] - 0s - loss: 0.0833     
Epoch 168/200
8806/8806 [==============================] - 0s - loss: 0.0833     - ETA: 0s - loss: 0.08
Epoch 169/200
8806/8806 [==============================] - 0s - loss: 0.0834     
Epoch 170/200
8806/8806 [==============================] - 0s - loss: 0.0833     
Epoch 171/200
8806/8806 [==============================] - 0s - loss: 0.0833     
Epoch 172/200
8806/8806 [==============================] - 0s - loss: 0.0832     
Epoch 173/200
8806/8806 [==============================] - 0s - loss: 0.0834     - ETA: 0s - loss: 0.0
Epoch 174/200
8806/8806 [==============================] - 0s - loss:

...

8806/8806 [==============================] - 0s - loss: 3.2233e-04     
Epoch 2/200
8806/8806 [==============================] - 0s - loss: 2.7259e-04     
Epoch 3/200
8806/8806 [==============================] - 0s - loss: 2.6559e-04     
Epoch 4/200
8806/8806 [==============================] - 0s - loss: 2.5830e-04     
Epoch 5/200
8806/8806 [==============================] - 0s - loss: 2.4445e-04     
Epoch 6/200
8806/8806 [==============================] - 0s - loss: 2.2772e-04     
Epoch 7/200
8806/8806 [==============================] - 0s - loss: 2.1167e-04     
Epoch 8/200
8806/8806 [==============================] - 0s - loss: 1.9720e-04     
Epoch 9/200
8806/8806 [==============================] - 0s - loss: 1.8718e-04     
Epoch 10/200
8806/8806 [==============================] - 0s - loss: 1.8263e-04     
Epoch 11/200
8806/8806 [==============================] - 0s - loss: 1.8000e-04     
Epoch 12/200
8806/8806 [==============================] - 0s - loss: 1.8015e-04     
E

8806/8806 [==============================] - 0s - loss: 1.5168e-04     
Epoch 97/200
8806/8806 [==============================] - 0s - loss: 1.5206e-04     
Epoch 98/200
8806/8806 [==============================] - 0s - loss: 1.5237e-04     
Epoch 99/200
8806/8806 [==============================] - 0s - loss: 1.5175e-04     
Epoch 100/200
8806/8806 [==============================] - 0s - loss: 1.5356e-04     
Epoch 101/200
8806/8806 [==============================] - 0s - loss: 1.5289e-04     
Epoch 102/200
8806/8806 [==============================] - 0s - loss: 1.5317e-04     
Epoch 103/200
8806/8806 [==============================] - 0s - loss: 1.5253e-04     
Epoch 104/200
8806/8806 [==============================] - 0s - loss: 1.5176e-04     
Epoch 105/200
8806/8806 [==============================] - 0s - loss: 1.5157e-04     
Epoch 106/200
8806/8806 [==============================] - 0s - loss: 1.5226e-04     
Epoch 107/200
8806/8806 [==============================] - 0s - loss: 1

8806/8806 [==============================] - 0s - loss: 1.5129e-04     
Epoch 192/200
8806/8806 [==============================] - 0s - loss: 1.5364e-04     
Epoch 193/200
8806/8806 [==============================] - 0s - loss: 1.5332e-04     
Epoch 194/200
8806/8806 [==============================] - 0s - loss: 1.5223e-04     
Epoch 195/200
8806/8806 [==============================] - 0s - loss: 1.5152e-04     
Epoch 196/200
8806/8806 [==============================] - 0s - loss: 1.5201e-04     
Epoch 197/200
8806/8806 [==============================] - 0s - loss: 1.5265e-04     
Epoch 198/200
8806/8806 [==============================] - 0s - loss: 1.5183e-04     
Epoch 199/200
8806/8806 [==============================] - 0s - loss: 1.5126e-04     
Epoch 200/200
8806/8806 [==============================] - 0s - loss: 1.5155e-04     
[[ 0.00239805]
 [ 0.02712784]
 [ 0.02469026]
 [ 0.02753253]
 [ 0.02818002]
 [ 0.00301208]
 [ 0.00218097]
 [ 0.02720878]
 [ 0.02688503]
 [ 0.02818002]
 [ 0

8806/8806 [==============================] - 0s - loss: 0.0061     
Epoch 41/200
8806/8806 [==============================] - 0s - loss: 0.0061     
Epoch 42/200
8806/8806 [==============================] - 0s - loss: 0.0061     
Epoch 43/200
8806/8806 [==============================] - 0s - loss: 0.0061     
Epoch 44/200
8806/8806 [==============================] - 0s - loss: 0.0061     
Epoch 45/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 46/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 47/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 48/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 49/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 50/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 51/200
8806/8806 [==============================] - 0s - loss: 0.0060     
Epoch 52/200
8806/8806 [=================

8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 140/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 141/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 142/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 143/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 144/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 145/200
8806/8806 [==============================] - 0s - loss: 0.0059       
Epoch 146/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 147/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 148/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 149/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 150/200
8806/8806 [==============================] - 0s - loss: 0.0059     
Epoch 151/200
8806/8806 [===

In [ ]:
hr_data.columns

In [ ]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on entire dataset till first row with null values after interpolation

#acc_dict={}

acc_1 ={}
acc_1 = nn_model(hr_data,acc_1,1)
print (acc_1)

In [ ]:
#pickle.dump(acc_dict,open('acc_dict','wb'))
acc_dict = pickle.load(open('acc_dict','rb'))

In [ ]:
'''
'Dishwasher' and 'Washing Machine' seem to be alright. Their accuracy remains similar.
But for the other attributes, it becomes negative at times. Why though? It should be something wrong with the way 
the data was interpolated(not using the graphical method) ?
On entire Dataset.

<->yr_data

'''
acc_dict

In [ ]:
#pickle.dump(acc_dict1,open('acc_dict1','wb'))
acc_dict1 = pickle.load(open('acc_dict1','rb'))

In [ ]:
'''
On the one-year dataset. hr_data
'''
acc_dict1

In [ ]:
# plotting the dishwasher load for every 24hrs
i = 0
while(i <= x.shape[0]):
    s = np.array(read1.values[i:i+1439,0])
    k = [t[-9:-7]+t[-6:-4] for t in s ]
    k = [int(p) for p in k]
    l = read.values[i:i+1439,0]
    plt.grid(True)
    plt.plot(k,l)
    plt.show()
    i+=1440

In [ ]:
d60 = pd.read_csv(path+"household_data_60min_singleindex.csv")
del d60['interpolated_values']
d60_cont1 = pd.DataFrame(d60.values[4893:8275])
d60_cont1.columns = d60.columns
del d60_cont1['cet_cest_timestamp']
print d60_cont1.shape

In [ ]:
del d60_cont1['DE_KN_residential4_grid_export']
del d60_cont1['DE_KN_residential4_pv']
d60_cont1.head()

In [ ]:
#Now try feeding this to the nn and see the results. This will help us in debugging what went wrong.
d60_cont1.isnull().sum(axis=0)

In [ ]:
#Testing the NN model on the continuous data without any interpolation.
acc_cont_dict={}

acc_cont_dict = nn_model(d60_cont1,acc_cont_dict,1)

print "The accuracies : ","\n\n",acc_cont_dict

In [ ]:
os.getlogin()